# US Stock Market Prediction (2014-2018)
**Authors:** David PAGNIEZ, Antoine KRYCHOWSKI, Gregoire MEHAH

## 1. Project Objective
The goal of this project is to analyze financial indicators of US companies from 2014 to 2018 and build a machine learning model to predict stock performance.

Our approach follows these steps:
1.  **Data Cleaning:** Handling missing values and outliers in financial data.
2.  **Baseline Model:** Attempting to predict the exact price variation (Regression).
3.  **Classification:** Switching to a trend prediction (Buy/Sell) if regression fails.
4.  **Optimization:** Using Feature Engineering and Ensemble methods to improve results.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PowerTransformer

# Visual configuration
plt.style.use("ggplot")
%matplotlib inline

# 1. Loading Data
# We iterate through the CSV files to merge them into a single dataframe.
years = [2014, 2015, 2016, 2017, 2018]
dfs = []

print("Loading data...")
for year in years:
    try:
        filename = f'{year}_Financial_Data.csv'
        df = pd.read_csv(filename)
        df['Year'] = year
        dfs.append(df)
        print(f"Loaded {filename}")
    except FileNotFoundError:
        print(f"Error: {filename} not found.")

data = pd.concat(dfs, ignore_index=True)

# 2. Target Consolidation
# The target column name is not consistent across files, so we merge them.
target_cols = [col for col in data.columns if 'PRICE VAR' in col]
data['Target_Price_Var'] = data[target_cols].bfill(axis=1).iloc[:, 0]
data['Target_Price_Var'] = pd.to_numeric(data['Target_Price_Var'], errors='coerce')

# Dropping useless columns
cols_to_drop = target_cols + ['Class', 'Unnamed: 0', 'Symbol']
data = data.drop(columns=[c for c in cols_to_drop if c in data.columns])

print(f"Initial Dataset Shape: {data.shape}")


Loading data...
Loaded 2014_Financial_Data.csv
Loaded 2015_Financial_Data.csv
Loaded 2016_Financial_Data.csv
Loaded 2017_Financial_Data.csv
Loaded 2018_Financial_Data.csv
Initial Dataset Shape: (22077, 225)


C:\Users\antoi\AppData\Local\Temp\ipykernel_24232\3272686016.py:34: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Target_Price_Var'] = data[target_cols].bfill(axis=1).iloc[:, 0]


## 2. Preprocessing and Outlier Management

Financial data is known to be very noisy. Some companies are massive (Apple, Google), while others are small. Additionally, some data points seem to be errors (e.g., a 10,000% increase).

If we leave these outliers, they will bias the model.
Strategy: We apply Winsorization (Clipping). We cap the values at the 1st and 99th percentiles to remove extreme outliers without deleting too much data.


In [3]:
# 1. Remove rows where the target is missing
data_clean = data.dropna(subset=['Target_Price_Var']).copy()

# 2. Remove columns with too many missing values (> 40%)
threshold = 0.4
data_clean = data_clean.dropna(thresh=len(data_clean) * (1 - threshold), axis=1)

# 3. Winsorization (Clipping)
# We limit the Target between -99% (almost bankruptcy) and +500% (x5)
data_clean['Target_Price_Var'] = data_clean['Target_Price_Var'].clip(lower=-99, upper=500)

# We limit the Features between the 1st and 99th quantile
numeric_cols = data_clean.select_dtypes(include=np.number).columns.drop(['Year', 'Target_Price_Var'])
lower_bounds = data_clean[numeric_cols].quantile(0.01)
upper_bounds = data_clean[numeric_cols].quantile(0.99)

data_clean[numeric_cols] = data_clean[numeric_cols].clip(lower=lower_bounds, upper=upper_bounds, axis=1)

print("Preprocessing done. Outliers have been clipped.")


Preprocessing done. Outliers have been clipped.
